# 🧠 Preparação de Dataset para Fine-Tuning

Este notebook realiza o seguinte processo:
- Verifica e instala a dependência `gdown`.
- Baixa o arquivo `trn.json` do Google Drive (caso não esteja presente).
- Processa o arquivo JSON e gera um arquivo `.jsonl` compatível com modelos de linguagem.
- Gera um relatório detalhado sobre as linhas processadas e ignoradas.

In [ ]:
# 📦 Instalação da dependência gdown (caso necessário)
import subprocess
import sys

try:
    import gdown
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "gdown"])
    import gdown

In [ ]:
# 📁 Verificação e download do arquivo JSON de entrada
import os

file_id = "1kq9bSDUfj0sxGO6C9bb95EYWtey6Smf_"
output = "trn.json"

if not os.path.exists(output):
    print("⬇️ Arquivo não encontrado. Iniciando download...")
    gdown.download(f"https://drive.google.com/uc?id={file_id}", output, quiet=False)
else:
    print("✅ Arquivo já existe. Pulando download.")

In [ ]:
# 🛠️ Função para preparar o dataset em formato jsonl
import json

def preparar_dataset_jsonl(caminho_entrada, caminho_saida):
    contador = 0
    ignorados_total = 0
    erros_json = 0
    titulos_vazios = 0
    descricoes_vazias = 0

    with open(caminho_entrada, 'r', encoding='utf-8') as f_in, \
         open(caminho_saida, 'w', encoding='utf-8') as f_out:

        for linha in f_in:
            try:
                item = json.loads(linha)
                titulo = item.get("title", "").strip()
                descricao = item.get("content", "").strip()

                if not titulo:
                    titulos_vazios += 1
                    ignorados_total += 1
                    continue

                if not descricao:
                    descricoes_vazias += 1
                    ignorados_total += 1
                    continue

                prompt = f"O que é {titulo}?"
                completion = descricao
                f_out.write(json.dumps({
                    "prompt": prompt,
                    "completion": completion
                }, ensure_ascii=False) + '\n')
                contador += 1

            except json.JSONDecodeError:
                erros_json += 1
                ignorados_total += 1

    print("✅ Dataset gerado com sucesso!")
    print(f"📄 Linhas válidas: {contador}")
    print(f"🚫 Linhas ignoradas: {ignorados_total}")
    print(f"   └─ 🧨 Erros de JSON: {erros_json}")
    print(f"   └─ ❌ Títulos vazios: {titulos_vazios}")
    print(f"   └─ ❌ Descrições vazias: {descricoes_vazias}")
    print(f"💾 Arquivo salvo em: {caminho_saida}")

In [ ]:
# 🚀 Execução da função de preparação de dataset
CAMINHO_JSON = "trn.json"
SAIDA_JSONL = "dataset_preparado.jsonl"

preparar_dataset_jsonl(CAMINHO_JSON, SAIDA_JSONL)